In [3]:
#!pip install --upgrade ortools

In [2]:
#restart the kernel if you installed ortools and it doesnt find the module
import ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import geopandas as gpd
import pandas as pd
import contextily as ctx
import pathlib
import numpy as np
import networkx
import osmnx as os
from matplotlib import pyplot as plt
from shapely.geometry import Point, Polygon, LineString
import shapely
import numpy as np

In [3]:
hospitals_gdf =  gpd.read_file('Hospitals_2024.geojson')
shelters_gdf = gpd.read_file('Shelters_2024.geojson')

In [4]:
game_grid = gpd.read_file('game_grid_2024.geojson')
game_grid['point'] = [game_grid['geometry'][i].centroid for i in range(0, len(game_grid))]

In [5]:
transport_network = networkx.read_graphml('transport_network_pre.graphml')

In [6]:
nodes_df = pd.DataFrame(list(transport_network.nodes))

In [7]:
def get_closest_node(point):
    #returns the name of the node closest to the point
    
    intersection = game_grid[game_grid.geometry.intersects(point)]
    node_key = intersection['MGRS']
    closest_node = nodes_df[nodes_df[0].isin(node_key)].iloc[0,0]
    
    return closest_node

In [22]:
def create_time_matrix(size):
    time_matrix = np.zeros((size,size))
    
    for row in range(size):
        for col in range(size):
            row_hospital = hospitals_gdf.iloc[row]
            col_hospital = hospitals_gdf.iloc[col]
    
            row_hospital_point = row_hospital['geometry'].centroid
            col_hospital_point = col_hospital['geometry'].centroid
    
            row_hospital_node = get_closest_node(row_hospital_point)
            col_hospital_node = get_closest_node(col_hospital_point)
    
            shortest_path_time = networkx.shortest_path_length(transport_network, row_hospital_node, 
                                  col_hospital_node, weight='travel_time')
    
            time_matrix[row, col] = shortest_path_time

    return time_matrix.astype(np.int64)

In [ ]:
time_matrix = create_time_matrix(17).tolist()

In [ ]:
def create_data_model(time_matrix):
    """Stores the data for the problem."""
    data = {}

    data["time_matrix"] = time_matrix
    
    data["demands"] = [0, 0, 500, 500, 200, 200, 200, 200, 200, 200, 200, 500, 500, 200, 200,200,500]
    data["vehicle_capacities"] = [1200,1200,1200,1200]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):

    
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_time = 0
    total_load = 0
    routes = []
    time = []
    for vehicle_id in range(data["num_vehicles"]):
        route = []
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_time = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index) #index, node index is the index of where the car is currently
            route.append(node_index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index)) #now index is index of where the car will travel
            route_time += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
                
        print(route)
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Time of the route: {route_time} min\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_time += route_time
        total_load += route_load
        routes.append(route)
    print(f"Total time of all routes: {total_time} min")
    print(f"Total load of all routes: {total_load}")
    print(routes)

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model(time_matrix) 

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.


    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()